# Deep Learning and Medical Imaging - Radiology

## Problem

We want to classify an image (patch) from an MR scan into one of 2 categories, {non-tumor,tumor}. Given such a classifier we could run it over all the image patches in an image to get a segmentation mask.

To do this we frame the problem as trying to estimate the conditional probabillity of the class given the image pixels, $f(x)=p(y|x)$, where $f(x)$ is the function we are trying to learn. For $f(x)$ to be a valid probabillity distribution we only require that $f_{i}(x)\ge 0$ and that $\sum_{i=0}^n f_{i} = 1$. A common trick in machine learning to convert any function into a probabillity distribution is to define $g_{i}(x) = \frac{e^{x_{i}}}{\sum_{j=0}^n e^{x_{j}}}$, $g$ is called the softmax function. By applying the softmax to a vector valued function we end up with a valid probabillity distribution.
We will use this to learn an "unconstrained" real valued function and apply a softmax to convert the output into a probabillity distribution.

## Imports and data loading

In [ ]:
# Run the below lines to download and unpack the data when running in Colab
# !wget -O data.tar https://github.com/fordanic/cmiv-ai-course/raw/master/data.tar
# !tar -xvf data.tar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout, Dense, Flatten
from keras.regularizers import l2
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras.callbacks import Callback
import tensorflow as tf

def get_device():
    device_string = '/cpu:0'
    gpu=None
    if gpu is not None:
        device_string='/device:GPU:{0}'.format(gpu)
    return tf.device(device_string)

def _init_keras():
    # Setup the session to dynamically allocate memory
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    K.set_session(session)

# Init the default session to be used by Keras
_init_keras()


In [ ]:
import os
import glob
import matplotlib.image
from sklearn.model_selection import train_test_split

def load_data():
    rootdir = os.path.abspath(os.getcwd())
    datadir = os.path.join(os.path.join(rootdir, "data"), "MR")
    pos_pattern = os.path.join(os.path.join(datadir,"positive_images"), "*.jpg")
    neg_pattern = os.path.join(os.path.join(datadir,"negative_images"), "*.jpg")
    pos_filenames = list(glob.glob(pos_pattern))
    neg_filenames = list(glob.glob(neg_pattern))

    pos_images = [matplotlib.image.imread(fname) for fname in pos_filenames]
    neg_images = [matplotlib.image.imread(fname) for fname in neg_filenames]
    X = np.vstack([np.array(pos_images, dtype=np.float32), np.array(neg_images, dtype=np.float32)])
    y = np.array([1]*len(pos_images) + [0]*len(neg_images), dtype=np.int32)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    return (X_train, y_train), (X_test, y_test)

In [ ]:
# This will download data and cache it.
(X_train, y_train), (X_test, y_test) = load_data()

Check the size of the training and test sets, aswell as the dimension of each array

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

We can visualize the image patches by plotting some of them

In [ ]:
def plot_patches(X, y, y_true=None, to_plot=None):    
    to_plot = to_plot or len(X)
    plt.figure(figsize=(16,8))
    for i in range(to_plot):
        plt.subplot(1, to_plot, i+1)
        plt.imshow(X[i].reshape((32, 32)), interpolation='nearest', cmap='gray')
        plt.text(0, 0, y[i], color='black', 
                 bbox=dict(facecolor='white', alpha=1))
        if y_true is not None:
            plt.text(0, 32, y_true[i], color='black', 
                     bbox=dict(facecolor='white', alpha=1))
            
        plt.axis('off')
    plt.show()

In [ ]:
plot_patches(X_train, y_train, to_plot=10)

## Keras models

We will be using Keras (https://keras.io/) with the TensorFlow (https://www.tensorflow.org/) backend to explore some of the concepts in deep learning.

The idea of Keras (and TensorFlow and Theano which are the two backend supported by Keras) is to represent your machine learning problem as a computation/data flow graph. The graph is defined in some language (tyically Python) and then the graph is compiled into efficient operations (typically written in C++/CUDA) and executed on either CPU or GPU.
To compose the graph, Keras supplies a number of building blocks that can be put together to express most graphs of interest. All the imports at the top from `keras.layers` specify the building blocks that we will use in the exercise.
Let's start by defining a very simple model for classifying these image patches.

In [ ]:
def build_mlp_model():
    # Images are 1 channel and 32x32
    input = Input(shape=(32, 32, 1))
    # Flatten the image into a 784-dimensional vector
    x = Flatten()(input)
    # Multiply the image-vector with a (32x32)x50 matrix and add a bias
    # Then apply an elementwise ReLU (max(0, x))
    x = Dense(50, activation='relu')(x)
    # The output from the previous layer is a 50-dimensional vector
    # for each image example. Use this vector to categorize the image
    # as one of the classes 0-1
    x = Dense(2)(x)
    # Return a model that represents the computation graph that maps
    # input to the 10 class scores.
    return Model(input, x)

The model returned from this function will take the role of $f(x)$ in the problem description. To construct a learning problem we also need a cost function that can measure how well a certain $f(x)$ approximates the true $p(y|x)$. 

## Cost function

We will be using the categorical cross-entropy cost function (https://en.wikipedia.org/wiki/Cross_entropy) when training the model.

The model defined above represents a computation that takes as input an image (or a bacth of images) and generates "scores" for each of the classes {0,1}, the scores represent the unnormalized probabillites for each class (the logits). To get a proper probabillity distribution we should apply a softmax to the values. We will skip this step since we are only interested in the relative size of the score for different classes (i.e. which class has the highest score). Naively applying a softmax when using categorical cross-entropy as cost function could also introduce numerical instabillity (taking the gradient of the log of a softmax). To circumvent this, but still use the categorical cross-entropy we can "cheat" a bit and directly use TensorFlows built in cost function for this scenario.

In [ ]:
def crossentropy_logits(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

## Optimization

The last part in defining the learning problem is to define how we should update the parameters of the model given the training data. We know that we should compute the gradient of the loss function with respect to the parameters of the model and move the parameters in the opposite direction. Computing the gradient by hand can be really tricky, fortunately Keras, TensorFlow and Theano all come with automatic differentiation. Using the chain-rule of calculus, the gradient with respect to the model parameters can be computed automatically. There are also different schemes for using the gradient to compute how to update the parameters, all this comes pre-packaged in Keras Optimizers. We will start with the workhorse of optimization, Stochastic Gradient Descent, SGD.

In [ ]:
with get_device():
    # Build model
    model = build_mlp_model()
    # Use Stochastic Gradient Descent as optimizer
    optimizer = SGD(lr=0.003)
    # Compile the model, giving it our custom loss-function
    model.compile(optimizer=optimizer, 
                  loss=crossentropy_logits, 
                  metrics=['accuracy'])

## Keras dataformat

In Keras an image is represented as a 3D tensor, width x height x channels. The image patches on disk represents the image only as width x height. The labels in the data are represented as plain numbers ({0,1}). When using cross-entropy in Keras the target distribution should be a "1-hot" encoding of the label (this corresponds to a target distribution where all probabillity is placed on the correct label).
To fix this we define a function that converts the data from the plain data format to a format the can be used within Keras.

In [ ]:
def to_tensors(X, y):
    # Convert X into a 4D tensor and y into a 2D tensor with 1-hot encoding
    return X[:, :, :, np.newaxis], to_categorical(y, num_classes=2)

In [ ]:
X_train, y_train = to_tensors(X_train, y_train)
X_test, y_test = to_tensors(X_test, y_test)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

To be able to track the progress of the learning we create a callback object that will log some metrics during training.

In [ ]:
class LogCallback(Callback):            
    def on_epoch_end(self, epoch, logs=None):                                        
        print("{}: L: {:.7} A: {:.7} VL: {:.7} VA: {:.7}".format(epoch,                                                                            
                                                                 logs['loss'], 
                                                                 logs['acc'], 
                                                                 logs['val_loss'], 
                                                                 logs['val_acc']))

## Training

To train the model and visualize the result we will call fit() on the model. We can start by training on only a fraction of the data. This is helpfull to spot any errors or parameters that might need tuning before investing more time training on the full dataset. (Set verbose=0 otherwise the browser might hang due to a lot of output.)

In [ ]:
# If the following lines produces an error saying ValueError,
# try uncommenting the following lines and run this cell again

#def crossentropy_logits(y_true, y_pred):
#    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

# If this made a difference, you have to go through all the code below and swap places on 
# the variables y_true and y_pred when calling K.categorical_crossentropy

# Use the first 5000 examples to train on
n_train = 5000
with get_device():
    # Fit the model and save the logs
    logs = model.fit(X_train[:n_train], y_train[:n_train], 
                     validation_split=0.3, 
                     epochs=50, 
                     verbose=0, 
                     callbacks=[LogCallback()])
# Plot the accuracy on the training data and the validation data
plt.plot(logs.history['acc'], c='r')
plt.plot(logs.history['val_acc'], c='g')
plt.show()

## Evaluation

Until now we have only evaluated the model on the training data (the validation data is part of the development). To evaluate how well the model can be expected to perform we should always test it on the test set. The test set SHOULD NOT be used to tune any parameters. The test set is used as the last verification of the model, we can also use it to explore what type of errors the model does.

In [ ]:
# Predict on test data
y_proba_test = model.predict(X_test)
# Compute the class with the highest score for each example
y_pred_test = np.argmax(y_proba_test, axis=-1)
# Compute the error vector
errors = y_pred_test != np.argmax(y_test, axis=-1)
# Compute the accuracy    
print("Accuracy: {}".format(1.0-np.mean(errors)))

# Plot the first examples.
to_evaluate = 5
X_eval = X_test[:to_evaluate]    
y_eval = y_pred_test[:to_evaluate]
plot_patches(X_eval, y_eval)

# Plot the first error examples
X_eval = X_test[np.where(errors)][:to_evaluate]
y_eval = y_pred_test[np.where(errors)][:to_evaluate]
plot_patches(X_eval, y_eval)

___
## Exercise

In [ ]:
# Try different values of the learning rate (tip: Start by looking at a log-scale 0.1, 0.01 ...)
# Try an optimizer with adaptive learning rate, Adam()

In [ ]:
# Now train with all data (tip: Setting n_train=-1 will slice all the data)

In [ ]:
# Increase the size of the hidden layer

In [ ]:
# Add a second hidden layer

In [ ]:
# Add a dropout regularization layer (x = Dropout(0.5)(x))

___

## Full code (example)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Lambda, Activation, BatchNormalization, Dropout, Dense, Flatten
from keras.regularizers import l2
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
from keras.callbacks import Callback
import os
import glob
import matplotlib.image
from sklearn.model_selection import train_test_split
import tensorflow as tf

def get_device():
    device_string = '/cpu:0'
    gpu=None
    if gpu is not None:
        device_string='/device:GPU:{0}'.format(gpu)
    return tf.device(device_string)

def _init_keras():
    # Setup the session to dynamically allocate memory
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    K.set_session(session)

# Init the default session to be used by Keras
_init_keras()

def load_data():
    rootdir = os.path.abspath(os.getcwd())
    datadir = os.path.join(os.path.join(rootdir, "data"), "MR")
    pos_pattern = os.path.join(os.path.join(datadir,"positive_images"), "*.jpg")
    neg_pattern = os.path.join(os.path.join(datadir,"negative_images"), "*.jpg")
    pos_filenames = list(glob.glob(pos_pattern))
    neg_filenames = list(glob.glob(neg_pattern))

    pos_images = [matplotlib.image.imread(fname) for fname in pos_filenames]
    neg_images = [matplotlib.image.imread(fname) for fname in neg_filenames]
    X = np.vstack([np.array(pos_images, dtype=np.float32), np.array(neg_images, dtype=np.float32)])
    y = np.array([1]*len(pos_images) + [0]*len(neg_images), dtype=np.int32)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    return (X_train, y_train), (X_test, y_test)


def crossentropy_logits(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

def build_mlp_model():
    # Images are 1 channel and 32x32
    input = Input(shape=(32, 32, 1))
    x = Flatten()(input)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)    
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)    
    x = Dense(2)(x)
    return Model(input, x)

def to_tensors(X, y):
    return X[:, :, :, np.newaxis], to_categorical(y, num_classes=2)

def plot_patches(X, y, y_true=None, to_plot=None):    
    to_plot = to_plot or len(X)
    plt.figure(figsize=(16,8))
    for i in range(to_plot):
        plt.subplot(1, to_plot, i+1)
        plt.imshow(X[i].reshape((32, 32)), interpolation='nearest', cmap='gray')
        plt.text(0, 0, y[i], color='black', 
                 bbox=dict(facecolor='white', alpha=1))
        if y_true is not None:
            plt.text(0, 32, y_true[i], color='black', 
                     bbox=dict(facecolor='white', alpha=1))
            
        plt.axis('off')
    plt.show()

class LogCallback(Callback):            
    def on_epoch_end(self, epoch, logs=None):                                        
        print("{}: L: {:.7} A: {:.7} VL: {:.7} VA: {:.7}".format(epoch,                                                                            
                                                                 logs['loss'], 
                                                                 logs['acc'], 
                                                                 logs['val_loss'], 
                                                                 logs['val_acc']))    
    
(X_train, y_train), (X_test, y_test) = load_data()
plot_patches(X_train, y_train, to_plot=10)

X_train, y_train = to_tensors(X_train, y_train)
X_test, y_test = to_tensors(X_test, y_test)

with get_device():
    model = build_mlp_model()

    optimizer = Adam()
    model.compile(optimizer=optimizer, loss = crossentropy_logits, metrics=['accuracy'])

    n_train = -1
    logs = model.fit(X_train[:n_train], y_train[:n_train], 
                     validation_split=0.3, 
                     epochs=5,
                     verbose=0, callbacks=[LogCallback()])
plt.plot(logs.history['acc'])
plt.plot(logs.history['val_acc'])
plt.show()

# Predict on test data
y_proba_test = model.predict(X_test)
y_pred_test = np.argmax(y_proba_test, axis=-1)
y_true = np.argmax(y_test, axis=-1)
errors = y_pred_test != y_true

# Compute the accuracy    
print("Accuracy: {}".format(1.0-np.mean(errors)))

# Plot the first examples.
to_evaluate = 15
X_eval = X_test[:to_evaluate]    
y_eval = y_pred_test[:to_evaluate]
plot_patches(X_eval, y_eval, y_true=y_true[:to_evaluate])

# Plot the first error examples
X_eval = X_test[np.where(errors)][:to_evaluate]
y_eval = y_pred_test[np.where(errors)][:to_evaluate]
plot_patches(X_eval, y_eval)

## CNN

The model above was a standard feed-forward network where all nodes in a previous layer are connected to all nodes in the next. This strategy leads to an explosion of the number of parameters in the model when the size of the input image increases. When dealing with images we often use a hypothesis of stationarity in the image, pixels in a neighbourhood are correlated simliarilly across the entire image, the absolute coordinate (x,y) of a pixel does not influence its distribution. Using this hypothesis we can share weights across the image, thus reducing the total nbumber of parameters that need to be learned. This is the idea behind Convolutional Neural Networks.

In CNNs the parameters of the model is convolved across the image to produce feature maps. The feature maps produced in one layer can be used as input to another convolution layer in the same way as layers are stacked in a feed-forward network. To introduce some translation invariance into the model the output feature maps are pooled at certain stages. This effectively increases the receptive field of later convolutions, allowing them to "see" a larger part of the input.

Lets perform the same task as before, tumor classification, but using convolutional layers.

In [ ]:
def build_conv_model():
    # Images are 1 channel and 32x32
    input = Input(shape=(32, 32, 1))
    
    # Compute 32 feature maps by convolving with 3x3 kernels
    x = Conv2D(32, (3, 3), padding="same", activation='relu')(input)
    # Output is now 32x32x32
    # Pool it to produce 32x16x16 feature maps
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(2)(x)
    return Model(input, x)

In [ ]:
with get_device():
    # Build model
    model = build_conv_model()
    # Use Adam
    optimizer = Adam()
    # Compile the model, giving it our custom loss-function
    model.compile(optimizer=optimizer, loss = crossentropy_logits, metrics=['accuracy'])

In [ ]:
# Use the first 5000 examples to train on
n_train = 5000
with get_device():
    # Fit the model and save the logs
    logs = model.fit(X_train[:n_train], y_train[:n_train], 
                     validation_split=0.3, 
                     epochs=5,
                     verbose=0,
                     callbacks=[LogCallback()])
# Plot the accuracy on the training data and the validation data
plt.plot(logs.history['acc'], c='r')
plt.plot(logs.history['val_acc'], c='g')
plt.show()

___
## Exercise

In [ ]:
# REMEMBER: Try the model with a few samples (5000) then use the full training set.

In [ ]:
# Try different sizes of the kernels (use odd sizes, 3, 5, 7)

In [ ]:
# Try to add a second Conv-Pool layer

In [ ]:
# After the Flatten-layer add a normal Dense-layer

___

## Full code (example)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Lambda, Activation, BatchNormalization, Dropout, Dense, Flatten
from keras.regularizers import l2
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam, SGD
from keras.datasets import mnist
import os
import glob
import matplotlib.image
from sklearn.model_selection import train_test_split
import tensorflow as tf

def get_device():
    device_string = '/cpu:0'
    gpu=None
    if gpu is not None:
        device_string='/device:GPU:{0}'.format(gpu)
    return tf.device(device_string)

def _init_keras():
    # Setup the session to dynamically allocate memory
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    K.set_session(session)

# Init the default session to be used by Keras
_init_keras()
def load_data():
    rootdir = os.path.abspath(os.getcwd())
    datadir = os.path.join(os.path.join(rootdir, "data"), "MR")
    pos_pattern = os.path.join(os.path.join(datadir,"positive_images"), "*.jpg")
    neg_pattern = os.path.join(os.path.join(datadir,"negative_images"), "*.jpg")
    pos_filenames = list(glob.glob(pos_pattern))
    neg_filenames = list(glob.glob(neg_pattern))

    pos_images = [matplotlib.image.imread(fname) for fname in pos_filenames]
    neg_images = [matplotlib.image.imread(fname) for fname in neg_filenames]
    X = np.vstack([np.array(pos_images, dtype=np.float32), np.array(neg_images, dtype=np.float32)])
    y = np.array([1]*len(pos_images) + [0]*len(neg_images), dtype=np.int32)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    
    return (X_train, y_train), (X_test, y_test)

def crossentropy_logits(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred, from_logits=True)

def build_conv_model():
    # Images are 1 channel and 32x32
    input = Input(shape=(32, 32, 1))
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(2)(x)
    return Model(input, x)

def to_tensors(X, y):
    return X[:, :, :, np.newaxis], to_categorical(y, num_classes=2)

def plot_patches(X, y, y_true=None, to_plot=None):    
    to_plot = to_plot or len(X)
    plt.figure(figsize=(16,8))
    for i in range(to_plot):
        plt.subplot(1, to_plot, i+1)
        plt.imshow(X[i].reshape((32, 32)), interpolation='nearest', cmap='gray')
        plt.text(0, 0, y[i], color='black', 
                 bbox=dict(facecolor='white', alpha=1))
        if y_true is not None:
            plt.text(0, 32, y_true[i], color='black', 
                     bbox=dict(facecolor='white', alpha=1))
            
        plt.axis('off')
    plt.show()
    
class LogCallback(Callback):            
    def on_epoch_end(self, epoch, logs=None):                                        
        print("{}: L: {:.7} A: {:.7} VL: {:.7} VA: {:.7}".format(epoch,                                                                            
                                                                 logs['loss'], 
                                                                 logs['acc'], 
                                                                 logs['val_loss'], 
                                                                 logs['val_acc']))    
    
(X_train, y_train), (X_test, y_test) = load_data()
plot_patches(X_train, y_train, to_plot=10)

X_train, y_train = to_tensors(X_train, y_train)
X_test, y_test = to_tensors(X_test, y_test)

with get_device():
    model = build_conv_model()

    optimizer = Adam()
    model.compile(optimizer=optimizer, loss = crossentropy_logits, metrics=['accuracy'])

    n_train = -1
    logs = model.fit(X_train[:n_train], y_train[:n_train], 
                     validation_split=0.3, 
                     epochs=5,
                     verbose=0,
                     callbacks=[LogCallback()])
plt.plot(logs.history['acc'])
plt.plot(logs.history['val_acc'])
plt.show()

with get_device():
    # Predict on test data
    y_proba_test = model.predict(X_test)
y_pred_test = np.argmax(y_proba_test, axis=-1)
y_true = np.argmax(y_test, axis=-1)
errors = y_pred_test != y_true
# Compute the accuracy    
print("Accuracy: {}".format(1.0-np.mean(errors)))

# Plot the first examples.
to_evaluate = 15
X_eval = X_test[:to_evaluate]    
y_eval = y_pred_test[:to_evaluate]
plot_patches(X_eval, y_eval, y_true=y_true[:to_evaluate])

# Plot the first error examples
X_eval = X_test[np.where(errors)][:to_evaluate]
y_eval = y_pred_test[np.where(errors)][:to_evaluate]
plot_patches(X_eval, y_eval)

___